In [1]:
import json
import pandas as pd
from transformers import AutoTokenizer

/home/sibava/miniconda3/envs/psat5/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
bert_tokenizer = AutoTokenizer.from_pretrained('cl-tohoku/bert-base-japanese')

In [34]:
f = open('decoded_psa_closest.jsonl',mode='r')
df_psa = pd.read_json(f,orient='records',lines=True)

In [35]:
df_psa['result'] = 'tn'

In [36]:
for index,row in df_psa.iterrows():
	if(row['output_token'] == ''):
		if(row['case_type'] == 'null'):
			continue
		else:
			df_psa.at[index,'result'] = 'fn'
	else:
		gold_arguments = row['gold_arguments']
		subword_gold_argument = [bert_tokenizer.tokenize(argument)[-1] for argument in gold_arguments]
		if(bert_tokenizer.tokenize(row['output_token'])[-1] in subword_gold_argument):
		# if(row['output_token'] in row['gold_arguments']):
			df_psa.at[index,'result'] = 'tp'
		else:
			df_psa.at[index,'result'] = 'fp'

In [37]:
df_dict = {
'df_dep' : df_psa.query('case_type == "dep"'),
'df_intra' : df_psa.query('case_type == "intra"'),
'df_inter' : df_psa.query('case_type == "inter"'),
'df_dep_passive' : df_psa.query('case_type == "dep" and alt_type == "passive"'),
'df_intra_passive' : df_psa.query('case_type == "intra" and alt_type == "passive"'),
'df_exo' : df_psa.query('case_type == "exog" or case_type == "exo1" or case_type == "exo2"')
}

In [32]:
df_dict['df_intra'].query('result == "tp"')[0:50]

,output_token,gold_arguments,case_name,case_type,predicate,alt_type,input_tokens,result
51,化,[化],ga,intra,なって,active,村山連立政権の最重要政策課題に<extra_id_0>なって<extra_id_1>いる特殊...,tp
54,省,[省],ga,intra,統合する,active,村山連立政権の最重要政策課題になっている特殊法人の整理・合理化で、通産省は日本貿易振興会とア...,tp
76,ジェトロ,"[ジェトロ, 会]",o,intra,設立さ,passive,村山連立政権の最重要政策課題になっている特殊法人の整理・合理化で、通産省は日本貿易振興会とア...,tp
88,研,"[研, 所]",o,intra,設立さ,passive,村山連立政権の最重要政策課題になっている特殊法人の整理・合理化で、通産省は日本貿易振興会とア...,tp
108,相,[相],ga,intra,進める,active,村山連立政権の最重要政策課題になっている特殊法人の整理・合理化で、通産省は日本貿易振興会とア...,tp
111,相,[相],ga,intra,表明,active,村山連立政権の最重要政策課題になっている特殊法人の整理・合理化で、通産省は日本貿易振興会とア...,tp
141,首相,[首相],ga,intra,進める,active,村山富市首相が二十日召集の通常国会冒頭に行う施政方針演説の骨格が十三日、固まった。「改革から...,tp
144,首相,[首相],ga,intra,表明する,active,村山富市首相が二十日召集の通常国会冒頭に行う施政方針演説の骨格が十三日、固まった。「改革から...,tp
174,相,[相],ga,intra,行わ,passive,村山富市首相とさきがけ代表の武村正義蔵相による社会・さきがけ党首会談が、外遊中の武村蔵相の帰...,tp
177,会談,[会談],ga,intra,なった,active,村山富市首相とさきがけ代表の武村正義蔵相による社会・さきがけ党首会談が、外遊中の武村蔵相の帰...,tp


In [24]:
def print_scores(df_name:str,df):
	value_count = df['result'].value_counts()
	tp = value_count['tp']
	fp = value_count['fp']
	fn = value_count['fn']

	precision = tp/(tp + fp)
	recall = tp/(tp + fn)
	f1 = 2*recall*precision/(recall + precision)
	print(f'DataFrame = {df_name}\nprecision : {precision}\nrecall : {recall}\nf1 : {f1}\n')


In [38]:
for df_name,df in df_dict.items():
	print_scores(df_name,df)

DataFrame = df_dep
precision : 0.9122793096607816
recall : 0.9702928517174445
f1 : 0.9403922049771998

DataFrame = df_intra
precision : 0.6533905579399142
recall : 0.9179932465026531
f1 : 0.7634139003109017

DataFrame = df_inter
precision : 0.3818609513852588
recall : 0.8454861111111112
f1 : 0.5261073100468131

DataFrame = df_dep_passive
precision : 0.8821428571428571
recall : 0.9623376623376624
f1 : 0.920496894409938

DataFrame = df_intra_passive
precision : 0.5253164556962026
recall : 0.8706293706293706
f1 : 0.6552631578947369

DataFrame = df_exo
precision : 0.7528781793842034
recall : 0.9686531174646917
f1 : 0.8472431455257607

